In [1]:
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q
!pip install jiwer -q
!pip install sacrebleu -q
!pip install pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12=

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, Dataset, Audio, DatasetDict
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
import pandas as pd

In [ ]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

# preprocessing Tatoeba

In [4]:
train_dataset = load_dataset("ymoslem/Tatoeba-Translations",split="train", streaming=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

In [5]:
english_sentences = (dataset["sentence_tgt"] for dataset in train_dataset if dataset["lang_tgt"]=="eng")

In [7]:
# Example: print the first 5
lst_eng = []
limit = 85153
for i, sentence in enumerate(english_sentences):
    lst_eng.append(sentence)
    if i >= limit:
        break

In [8]:
len(lst_eng)

85154

In [11]:
eng_dataset = pd.DataFrame(lst_eng, columns=["text_en"])

In [12]:
eng_dataset.to_csv("eng_dataset.csv", index=False)

# translate

In [3]:
!pip3 install ctranslate2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.2 MB/s eta 0:00:00


In [4]:
df = pd.read_csv("eng_dataset.csv")

In [5]:
dataset = Dataset.from_pandas(df)

In [6]:
dataset

Dataset({
    features: ['text_en'],
    num_rows: 85154
})

In [7]:
dataset

Dataset({
    features: ['text_en'],
    num_rows: 85154
})

In [8]:
mt_model_name = "kreasof-ai/nllb-200-distilled-600M-en2bem"
output_mt = "ct2-nllb-200-distilled-600M-finetuned"
!ct2-transformers-converter --model {mt_model_name} --quantization float16 --output_dir {output_mt}

config.json: 100% 897/897 [00:00<00:00, 5.49MB/s]
2025-04-09 12:39:33.987446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744202374.063047    1237 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744202374.080095    1237 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
model.safetensors: 100% 2.46G/2.46G [00:32<00:00, 75.7MB/s]
generation_config.json: 100% 154/154 [00:00<00:00, 1.22MB/s]
tokenizer_config.json: 100% 39.9k/39.9k [00:00<00:00, 2.23MB/s]
sentencepiece.bpe.model: 100% 4.85M/4.85M [00:00<00:00, 13.7MB/s]
tokenizer.json: 100% 32.2M/32.2M [00:00<00:00, 39.4MB/s]
special_tokens_map.json: 100% 4.23k/4.23k [00:00<00:00, 29.1MB/s]


In [9]:
!wget https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model

--2025-04-09 12:40:51--  https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.227.160, 16.182.35.160, 3.5.9.153, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.227.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4852054 (4.6M) [binary/octet-stream]
Saving to: ‘flores200_sacrebleu_tokenizer_spm.model’

flores200_sacrebleu 100%[===================>]   4.63M  9.43MB/s    in 0.5s    

2025-04-09 12:40:52 (9.43 MB/s) - ‘flores200_sacrebleu_tokenizer_spm.model’ saved [4852054/4852054]



In [10]:
import os

directory = "/content/"

ct2_model_name = output_mt

ct_model_path = os.path.join(directory, ct2_model_name)
sp_model_path = os.path.join(directory, "flores200_sacrebleu_tokenizer_spm.model")

In [11]:
import ctranslate2
import sentencepiece as spm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

sp = spm.SentencePieceProcessor()
sp.load(sp_model_path)

translator = ctranslate2.Translator(ct_model_path, device=device)

In [12]:
# Translate

# src_lang = "ben_Beng"
# tgt_lang = "eng_Latn"

src_lang = "eng_Latn"
tgt_lang = "bem_Latn"

beam_size = 5

source_sentences = dataset["text_en"][:10]

source_sents = [sent.strip() for sent in source_sentences]
target_prefix = [[tgt_lang]] * len(source_sents)

# Subword the source sentences
source_sents_subworded = sp.encode_as_pieces(source_sents)
source_sents_subworded = [[src_lang] + sent + ["</s>"] for sent in source_sents_subworded]

# Translate the source sentences
translations = translator.translate_batch(source_sents_subworded,
                                          batch_type="tokens",
                                          max_batch_size=2024,
                                          beam_size=beam_size,
                                          target_prefix=target_prefix)
translations = [translation.hypotheses[0] for translation in translations]

# Desubword the target sentences
translations_desubword = sp.decode(translations)
translations_desubword = [sent[len(tgt_lang):].strip() for sent in translations_desubword]


print(*translations_desubword[:10], sep="\n")

Tekulanda saana, citafye ifingi.
ukwensha kwena cintu icisuma saana
awe ine nshikwetabila iyo.
Ndemona kwati apa ndesubila ndeya kusukulu.
Awe kwena kuti bakwipaya.
Baciya baleepaya ing'ombe shesu.
Kulebelenga ifitabo.
Awe kwena kuti banwipaya bonse mwe!
Bwamukwai, akasuba kasuma, icungulo, ubushiku ubusaka.
Tom tafwele insokoshi.


In [13]:
df_en_be = pd.DataFrame({"text_en": source_sents, "text_bem": translations_desubword})

In [14]:
df_en_be

,text_en,text_bem
0,"Talk less, do more.","Tekulanda saana, citafye ifingi."
1,Trying is something really beautiful.,ukwensha kwena cintu icisuma saana
2,I no longer need you.,awe ine nshikwetabila iyo.
3,I am going to school right now.,Ndemona kwati apa ndesubila ndeya kusukulu.
4,They'll kill you!,Awe kwena kuti bakwipaya.
5,They butchered our cows.,Baciya baleepaya ing'ombe shesu.
6,You ought to read books.,Kulebelenga ifitabo.
7,They'll kill you all!,Awe kwena kuti banwipaya bonse mwe!
8,Good morning. Good afternoon. Good evening. Go...,"Bwamukwai, akasuba kasuma, icungulo, ubushiku ..."
9,Tom was not wearing socks.,Tom tafwele insokoshi.


In [ ]:
df_en_be.to_csv("tatoeba_en_bem.csv", index=False)